<a href="https://colab.research.google.com/github/dvlpana/SparkPlatzi/blob/main/CursoSpakPlatzi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!apt-get install  openjdk-8-jdk-headless -qq > /dev/null
!ad-apt-repository ppa:openjdk-r/ppa
!apt-get update && sudo apt-get upgrade -y
!apt-get -y install openjdk-8-jre
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar -xvf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

/bin/bash: ad-apt-repository: command not found
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12

In [2]:
!pip install -q findspark

In [3]:
!ls

drive  sample_data  spark-3.1.1-bin-hadoop2.7  spark-3.1.1-bin-hadoop2.7.tgz


In [4]:
import time
print("Sleeping")
time.sleep(30) # sleep for a while; interrupt me!
print("Done Sleeping")

Sleeping
Done Sleeping


In [5]:
!ls

drive  sample_data  spark-3.1.1-bin-hadoop2.7  spark-3.1.1-bin-hadoop2.7.tgz


In [6]:
!head -n 15 /content/sample_data/files_curso_spark/data.csv

State,Color,Count
TX,Red,20
NV,Blue,66
CO,Blue,79
OR,Blue,71
WA,Yellow,93
WY,Blue,16
CA,Yellow,53
WA,Green,60
OR,Green,71
TX,Green,68
NV,Green,59
AZ,Brown,95
WA,Yellow,20
AZ,Blue,75


In [7]:
!spark-submit /content/sample_data/files_curso_spark/codeExample.py /content/sample_data/files_curso_spark/data.csv

21/06/11 12:33:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/06/11 12:33:40 INFO SparkContext: Running Spark version 3.1.2
21/06/11 12:33:40 INFO ResourceUtils: ==============================================================
21/06/11 12:33:40 INFO ResourceUtils: No custom resources configured for spark.driver.
21/06/11 12:33:40 INFO ResourceUtils: ==============================================================
21/06/11 12:33:40 INFO SparkContext: Submitted application: PythonMnMCount
21/06/11 12:33:40 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
21/06/11 12:33:40 INFO Resour

In [8]:
!ls

drive  sample_data  spark-3.1.1-bin-hadoop2.7  spark-3.1.1-bin-hadoop2.7.tgz


In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [10]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [11]:
archivo = './sample_data/files_curso_spark/data.csv'
df_spark = spark.read.csv(archivo, inferSchema=True, header=True)

# imprimir tipo de archivo
print(type(df_spark))

<class 'pyspark.sql.dataframe.DataFrame'>


¿Nombre de las Columnas de dataframe?

In [12]:
df_spark.columns

['State', 'Color', 'Count']

Estructura del dataframe

In [13]:
df_spark.printSchema()

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: integer (nullable = true)



Ver los primeros 20 registros del dataframe

In [14]:
df_spark.show()

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   20|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
|   WY|  Blue|   16|
|   CA|Yellow|   53|
|   WA| Green|   60|
|   OR| Green|   71|
|   TX| Green|   68|
|   NV| Green|   59|
|   AZ| Brown|   95|
|   WA|Yellow|   20|
|   AZ|  Blue|   75|
|   OR| Brown|   72|
|   NV|   Red|   98|
|   WY|Orange|   45|
|   CO|  Blue|   52|
|   TX| Brown|   94|
|   CO|   Red|   82|
+-----+------+-----+
only showing top 20 rows



Descricipcion Estadistica del dataframe

In [15]:
df_spark.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
State,99999,None,None,AZ,WY
Color,99999,None,None,Blue,Yellow
Count,99999,55.00090000900009,26.26648202084822,10,100


Descripcion estadistica de una sola columna (‘median_house_value’)

In [16]:
df_spark.describe(['color']).show()

+-------+------+
|summary| color|
+-------+------+
|  count| 99999|
|   mean|  null|
| stddev|  null|
|    min|  Blue|
|    max|Yellow|
+-------+------+



## 8 Transformaciones y Acciones

In [17]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

sc = spark.sparkContext
lines = sc.textFile("spark-2.4.4-bin-hadoop2.7/README.md")
sc



<SparkContext master=local[*] appName=pyspark-shell>

In [18]:
path = '/content/sample_data/files_curso_spark/'
equiposOlimpicosRDD = sc.textFile(path+'paises.csv') \
.map(lambda line: line.split(","))

# Muestra la data con take , evita utilizar collect()

In [19]:
equiposOlimpicosRDD.take(10)



[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR']]

9 Acciones de modificacion sobre RDDs

In [20]:
##Contar cuantas siglas tenemos
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

In [21]:
##Agrupación a partir de siglas y obtener cantidad de equipos por país
equiposOlimpicosRDD \
.map(lambda x: (x[2], x[1])) \
.groupByKey() \
.mapValues(len).take(5)

[('ARG', 18), ('AFG', 1), ('IRL', 7), ('POR', 21), ('AUS', 23)]

In [22]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [23]:
##Agrupación a partir de siglas y obtener cantidad de equipos por país
equiposOlimpicosRDD \
.map(lambda x: (x[2], x[1])) \
.groupByKey() \
.mapValues(list).take(3)

[('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan']),
 ('IRL',
  ['Akatonbo',
   'Aletta',
   'Ireland',
   'Ireland-1',
   'Ireland-2',
   'The Cloud',
   'Three Leaves'])]

In [24]:
##Obtener los datos sólo de un país o contenido con filter()
equiposArgentinos = equiposOlimpicosRDD.filter(lambda x: "ARG" in x)
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [25]:
## Cuenta todos los valores, nada recomendable para datos extensos
equiposOlimpicosRDD.count()

1185

In [26]:
##Cuenta  hasta el tiempo aproximado
equiposOlimpicosRDD.countApprox(20)

1185

10 Acciones de conteo sobre RDDs

In [27]:
## importamos ambos
deportistaOlimpicoRDD = sc.textFile(path+'deportista.csv') \
.map(lambda line: line.split(","))

deportistaOlimpicoRDD2 = sc.textFile(path+'deportista2.csv') \
.map(lambda line: line.split(","))

In [28]:
## Unimos los RDD con union(). Spark solo tiene esta operacion
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [29]:
deportistaOlimpicoRDD.count()

135572

In [30]:
##Ver contenido en equipos olimpicos
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [31]:
deportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [32]:
# seleccionamos las comumnas ID para hacer un join
deportistaOlimpicoRDD \
.map(lambda deportista: [deportista[-1],deportista[:-1]]) \
.join(equiposOlimpicosRDD.map(lambda equipo: [equipo[0], equipo [2]])) \
.take(6)

[('278', (['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0'], 'SWE')),
 ('278', (['86368', 'August Nilsson', '1', '27', '0', '0'], 'SWE')),
 ('278', (['112720', 'Gustaf Fredrik Sderstrm', '1', '34', '0', '0'], 'SWE')),
 ('278',
  (['114107', 'Karl Gustaf Vilhelm Staaf Johansson ', '1', '19', '0', '0'],
   'SWE')),
 ('278',
  (['130945', 'Charles Gustav Wilhelm Winckler', '1', '33', '0', '0'], 'SWE')),
 ('982', (['51', 'Nstor Abad Sanjun', '1', '23', '167', '64'], 'ESP'))]

In [33]:
# utilizamos takeSample (repetidos, muestra, semilla)
deportistaOlimpicoRDD \
.map(lambda deportista: [deportista[-1],deportista[:-1]]) \
.join(equiposOlimpicosRDD.map(lambda equipo: [equipo[0], equipo [2]])) \
.takeSample(False,6,25)

[('514', (['58198', 'Mayumi Kawasaki', '2', '24', '167', '52'], 'JPN')),
 ('399',
  (['129907', 'Armin Christian Weyrauch', '1', '28', '193', '95'], 'GER')),
 ('487', (['53275', 'Khashaba Dadasaheb Jahdav', '1', '21', '0', '0'], 'IND')),
 ('138',
  (['110517', 'Hlio Frederico Gomes da Silva', '1', '22', '183', '79'],
   'BRA')),
 ('507', (['121829', 'Nicola Trentin', '1', '30', '178', '70'], 'ITA')),
 ('138',
  (['23297', 'Kissya Cataldo da Costa', '2', '30', '180', '67'], 'BRA'))]

In [34]:
resultado = sc.textFile(path+'resultados.csv') \
.map(lambda line: line.split(","))

In [35]:
resultadoGanador = resultado.filter(lambda w: 'NA' not in w[1])

In [36]:
resultadoGanador.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14']]

In [37]:
##Reto Hacer un join con equipos y deportista para obtener valores importantes
deportistaOlimpicoRDD \
.map(lambda deportista: [deportista[-1],deportista[:-1]]) \
.join(equiposOlimpicosRDD.map(lambda equipo: [equipo[0], equipo [2]])) \
.join(resultadoGanador.map(lambda resultado: [resultado[2], resultado [1]])) \
.takeSample(False, 6, 25)


[('93',
  ((['73927', 'Yana Eduardovna Maksimova', '2', '19', '182', '70'], 'BLR'),
   'Gold')),
 ('705',
  ((['121086', 'Linda Toorop Moor ', '2', '16', '165', '53'], 'NED'), 'Gold')),
 ('1071',
  ((['118844', 'Nurcan Taylan', '2', '20', '151', '48'], 'TUR'), 'Silver')),
 ('705',
  ((['27539', 'Arnold Pieter Bernard Arno den Hartog', '1', '29', '187', '88'],
    'NED'),
   'Gold')),
 ('705',
  ((['66236', 'Frederik Joseph Frits Lamp', '1', '18', '0', '0'], 'NED'),
   'Bronze')),
 ('1181',
  ((['17015', 'Hastings Bwalya', '1', '23', '168', '64'], 'ZAM'), 'Gold'))]

Clase 11 Solucion reto deportistas

In [38]:
#esta clase muestra la solucion del profesor al reto.
deportistaPaises = deportistaOlimpicoRDD \
    .map(lambda l: [l[-1], l[:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]]))

deportistaPaises.join(resultadoGanador).take(6)


[('972',
  ((['3585', 'An JaeHyeong', '1', '23', '183', '74'], 'KOR'), 'Silver')),
 ('972', ((['19788', 'Chang YeNa', '2', '26', '172', '61'], 'KOR'), 'Silver')),
 ('972',
  ((['40891', 'Go SeongHyeon', '1', '25', '182', '85'], 'KOR'), 'Silver')),
 ('972',
  ((['44419', 'Ha JeongEun', '2', '21', '173', '66'], 'KOR'), 'Silver')),
 ('972',
  ((['45476', 'Han SangHun', '1', '23', '175', '79'], 'KOR'), 'Silver')),
 ('972',
  ((['51378', 'Hwang HyeYeong', '2', '26', '0', '0'], 'KOR'), 'Silver'))]

#Clase 12 Operaciones Numéricas
##En esta clase hacemos el encoding de los valores de las medallas y obtenemos los puntos que tiene cada país.
Acciones de Conteo sobre RDD

```
# Tiene formato de código
```



In [39]:
!ls ./sample_data/files_curso_spark/

codeExample.py	deportista2.csv      evento.csv		    paises.csv
data.csv	deportista.csv	     juegos.csv		    resultados.csv
deporte.csv	deportistaError.csv  modelo_relacional.jpg


In [40]:
#Almacenamos el puntaje por medalla
valoresMedallas = {'Gold'  :7,
                   'Silver':5,
                   'Bronze':4}

In [41]:
paisesMedallas = deportistaPaises.join(resultadoGanador)



In [42]:
paisesMedallas = paisesMedallas \
    .map(lambda x: (x[1][0][-1], valoresMedallas[x[1][1] ] ) )

In [43]:
paisesMedallas.take(5)

[('KOR', 5), ('KOR', 5), ('KOR', 5), ('KOR', 5), ('KOR', 5)]

In [44]:
from operator import add

conclusion = paisesMedallas.reduceByKey((add)) \
    .sortBy(lambda x:x[1], ascending=False)

In [45]:
conclusion.take(10)

[('CAN', 32538),
 ('ARG', 12520),
 ('HUN', 10860),
 ('MEX', 6124),
 ('RSA', 3788),
 ('BLR', 3580),
 ('LTU', 1535),
 ('MGL', 1460),
 ('USA', 1342),
 ('AZE', 1218)]

Clase 13 Creación de DataFrames

In [46]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.types import Row

from pyspark.sql import SQLContext





In [47]:
#spark = SparkContext(master='local', appName="DataFrames")
sqlContext = SQLContext(spark)




In [48]:
!ls ./sample_data/files_curso_spark/

codeExample.py	deportista2.csv      evento.csv		    paises.csv
data.csv	deportista.csv	     juegos.csv		    resultados.csv
deporte.csv	deportistaError.csv  modelo_relacional.jpg


In [49]:
!head -n 10 ./sample_data/files_curso_spark/juegos.csv

,nombre_juego,annio,temporada,ciudad
1,1896 Verano,1896,Verano,Athina
2,1900 Verano,1900,Verano,Paris
3,1904 Verano,1904,Verano,St. Louis
4,1906 Verano,1906,Verano,Athina
5,1908 Verano,1908,Verano,London
6,1912 Verano,1912,Verano,Stockholm
7,1920 Verano,1920,Verano,Antwerpen
8,1924 Invierno,1924,Invierno,Chamonix
9,1924 Verano,1924,Verano,Paris


In [50]:
spark

In [51]:
# Paso 1 crear un dataFrame es crearun Schema como una base de datos
path = '/content/sample_data/files_curso_spark/'
juegoSchema = StructType([
     StructField('juego_id', IntegerType(), False),
     StructField('anio', StringType(), False),
     StructField('temporada', StringType(), False),
     StructField('ciudad', StringType(), False),
    ])
# Paso 2 Crea Data Frame con la Lectura del Schema 
juegoDF = sqlContext.read.schema(juegoSchema) \
          .option('header', 'true').csv(path+'resultados.csv')

In [52]:
juegoDF.show(4)

+--------+----+---------+------+
|juego_id|anio|temporada|ciudad|
+--------+----+---------+------+
|       1|  NA|        1|    39|
|       2|  NA|        2|    49|
|       3|  NA|        3|     7|
|       4|Gold|        4|     2|
+--------+----+---------+------+
only showing top 4 rows



Clase 14 Inferencia de tipos de datos

In [53]:
deportistaOlimpicoRDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [54]:
# Retiramos encabezados
def eliminarEncabezado(indice, iterador):
  return iter(list(iterador)[1:])

In [55]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.\
mapPartitionsWithIndex(eliminarEncabezado)

In [56]:
deportistaOlimpicoRDD.take(5)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [57]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6]),
))

In [58]:
deportistaOlimpicoRDD.take(5)

[(1, 'A Dijiang', 1, 24, 180, 80.0, 199),
 (2, 'A Lamusi', 1, 23, 170, 60.0, 199),
 (3, 'Gunnar Nielsen Aaby', 1, 24, 0, 0.0, 273),
 (4, 'Edgar Lindenau Aabye', 1, 34, 0, 0.0, 278),
 (5, 'Christine Jacoba Aaftink', 2, 21, 185, 82.0, 705)]

In [59]:
schema_deportistaOlimpico = StructType([
    StructField('deportista_id', IntegerType(),False),
    StructField('nombre', StringType(),False),
    StructField('genero', IntegerType(),False),
    StructField('edad', IntegerType(),False),
    StructField('altura', IntegerType(),False),
    StructField('peso', FloatType(),False),
    StructField('equipo_id', IntegerType(),False)
]    
)

In [60]:
deportistaDF =  sqlContext.createDataFrame(deportistaOlimpicoRDD, schema_deportistaOlimpico)

In [61]:
deportistaDF.show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



Retos crear los demas DataFrames testantes a partir de los RDD

In [62]:
# Países  equipos, sc???
paisesRDD = sc.textFile(path+'paises.csv') \
.map(lambda line: line.split(",")) \
.mapPartitionsWithIndex(eliminarEncabezado) \
.map(lambda l:(
    int(l[0]),
    l[1],
    l[2]
))

schema = StructType([
    StructField('pais_id', IntegerType(),False),
    StructField('equipo', StringType(),False),
    StructField('sigla', StringType(),False)

]    
)

paisesDF =  sqlContext.createDataFrame(paisesRDD, schema)
paisesDF.show(5)

+-------+--------------------+-----+
|pais_id|              equipo|sigla|
+-------+--------------------+-----+
|      1|         30. Februar|  AUT|
|      2|A North American ...|  MEX|
|      3|           Acipactli|  MEX|
|      4|             Acturus|  ARG|
|      5|         Afghanistan|  AFG|
+-------+--------------------+-----+
only showing top 5 rows



In [63]:
resultados_schema = StructType([
    StructField("resultado_id", IntegerType(), False),
    StructField("medalla", StringType(), False),
    StructField("deportista_id", IntegerType(), False),
    StructField("juego_id", IntegerType(), False),
    StructField("evento_id", IntegerType(), False)
])

resultadosDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(resultados_schema).\
            load(path+"resultados.csv")

resultadosDF.show(4)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
+------------+-------+-------------+--------+---------+
only showing top 4 rows



In [64]:
evento_schema = StructType([
    StructField("evento_id", IntegerType(), False),
    StructField("evento", StringType(), False), 
    StructField("deporte_id", IntegerType(), False)
])

eventoDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(evento_schema).\
        load(path+"evento.csv")

eventoDF.show(4)

+---------+--------------------+----------+
|evento_id|              evento|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
+---------+--------------------+----------+
only showing top 4 rows



In [65]:
deporte_schema = StructType([
          StructField("deporte_id",IntegerType(),False),
          StructField("deporte", StringType(),False)
])

deporteDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(deporte_schema).\
        load(path+"deporte.csv")

deporteDF.show(4)

+----------+----------+
|deporte_id|   deporte|
+----------+----------+
|         1|Basketball|
|         2|      Judo|
|         3|  Football|
|         4|Tug-Of-War|
+----------+----------+
only showing top 4 rows



In [66]:
# juegos

juegos_schema = StructType([
    StructField('nombre_juego', StringType(),False),
    StructField('annio', IntegerType(),False),
    StructField('temporada', StringType(),False),
    StructField('ciudad', StringType(),False)
]    
)

juegosDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(juegos_schema).\
        load(path+"juegos.csv")

juegosDF.show(4)


+------------+-----+---------+------+
|nombre_juego|annio|temporada|ciudad|
+------------+-----+---------+------+
|           1| null|     1896|Verano|
|           2| null|     1900|Verano|
|           3| null|     1904|Verano|
|           4| null|     1906|Verano|
+------------+-----+---------+------+
only showing top 4 rows



 Clase 15 Operaciones sobre DF.
En esta clase aprendemos como obtener el schema de un DataFrame, a renombrar columnas y select().

In [67]:
# Ver el esquema
deporteDF.printSchema()

root
 |-- deporte_id: integer (nullable = true)
 |-- deporte: string (nullable = true)



In [68]:
# Ver el esquema
deportistaDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- altura: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [69]:
#Renombrar columnas
deportistaOlimpicoDF = deportistaDF \
.withColumnRenamed("genero", 'sexo') \
.drop('altura')


In [70]:
deportistaOlimpicoDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- sexo: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [71]:
# Usando el rey SELECT
from pyspark.sql.functions import *
deportistaOlimpicoDF = deportistaOlimpicoDF.select("deportista_id","nombre",
 col("edad").alias("edadAlJugar"),
 "equipo_id"
 )

In [72]:
deportistaOlimpicoDF.show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|            1|           A Dijiang|         24|      199|
|            2|            A Lamusi|         23|      199|
|            3| Gunnar Nielsen Aaby|         24|      273|
|            4|Edgar Lindenau Aabye|         34|      278|
|            5|Christine Jacoba ...|         21|      705|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [73]:
# Usando filter
deportistaOlimpicoDF = deportistaOlimpicoDF.filter(
    (deportistaOlimpicoDF.edadAlJugar != 0)
)

In [74]:
deportistaOlimpicoDF.sort("edadAlJugar").show()

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|        71691|  Dimitrios Loundras|         10|      333|
|        76675|   Marcelle Matthews|         11|      967|
|       118925|Megan Olwen Deven...|         11|      413|
|        40129|    Luigina Giavotti|         11|      507|
|        37333|Carlos Bienvenido...|         11|      982|
|        51268|      Beatrice Hutiu|         11|      861|
|        22411|Magdalena Cecilia...|         11|      413|
|        52070|        Etsuko Inada|         11|      514|
|        70616|          Liu Luyang|         11|      199|
|        47618|Sonja Henie Toppi...|         11|      742|
|       126307|        Liana Vicens|         11|      825|
|        54620|Belita Gladys Lyn...|         12|      413|
|        25877|Olga Lucia de Ang...|         12|      225|
|        59727|Marika Kilius Zah...|         12|      39

Clase 16 Agrupaciones y operaciones join sobre DF


In [75]:
#vizualizamos los esquemas de los DF
deportistaOlimpicoDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- edadAlJugar: integer (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [76]:
resultadosDF.printSchema()

root
 |-- resultado_id: integer (nullable = true)
 |-- medalla: string (nullable = true)
 |-- deportista_id: integer (nullable = true)
 |-- juego_id: integer (nullable = true)
 |-- evento_id: integer (nullable = true)



In [77]:
juegosDF.printSchema()

root
 |-- nombre_juego: string (nullable = true)
 |-- annio: integer (nullable = true)
 |-- temporada: string (nullable = true)
 |-- ciudad: string (nullable = true)



In [78]:
deporteDF.printSchema()

root
 |-- deporte_id: integer (nullable = true)
 |-- deporte: string (nullable = true)



In [79]:
paisesDF.printSchema()

root
 |-- pais_id: integer (nullable = false)
 |-- equipo: string (nullable = false)
 |-- sigla: string (nullable = false)



In [80]:
#JOINS
deportistaOlimpicoDF \
.join(
    resultadosDF,
    deportistaOlimpicoDF.deportista_id == resultadosDF.deportista_id,
    "left"
) \
.join(
    juegoDF,
    juegoDF.juego_id == resultadosDF.juego_id,
) \
.join(
    eventoDF,
    eventoDF.evento_id == resultadosDF.evento_id,
    "left"
) \
.select(deportistaOlimpicoDF.nombre,
        "edadAlJugar",
        "medalla",
        col("anio").alias("annio de juego"),
        deportistaOlimpicoDF.nombre.alias("Nombre disciplina")   
).show()



+--------------------+-----------+-------+--------------+--------------------+
|              nombre|edadAlJugar|medalla|annio de juego|   Nombre disciplina|
+--------------------+-----------+-------+--------------+--------------------+
|           A Dijiang|         24|     NA|        Bronze|           A Dijiang|
|            A Lamusi|         23|     NA|          Gold|            A Lamusi|
| Gunnar Nielsen Aaby|         24|     NA|            NA| Gunnar Nielsen Aaby|
|Edgar Lindenau Aabye|         34|   Gold|            NA|Edgar Lindenau Aabye|
|Christine Jacoba ...|         21|     NA|            NA|Christine Jacoba ...|
|Christine Jacoba ...|         21|     NA|            NA|Christine Jacoba ...|
|Christine Jacoba ...|         21|     NA|        Bronze|Christine Jacoba ...|
|Christine Jacoba ...|         21|     NA|        Bronze|Christine Jacoba ...|
|Christine Jacoba ...|         21|     NA|            NA|Christine Jacoba ...|
|Christine Jacoba ...|         21|     NA|          

In [81]:
#Reto hacer un JOIN con todas las medallas ganadoras
#unidas con el país y el equipo al que pertenecen estas medallas
resultadosDF \
.join(
    deportistaOlimpicoDF,
   resultadosDF.deportista_id == deportistaOlimpicoDF.deportista_id,
    "left"
) \
.join(
   paisesDF,
    deportistaOlimpicoDF.equipo_id == paisesDF.pais_id ,
    "left"
) \
.select("medalla",
        paisesDF.equipo,
       paisesDF.sigla
) \
.where (resultadosDF.medalla != "NA").show()




+-------+------+-----+
|medalla|equipo|sigla|
+-------+------+-----+
| Silver| Hi-Hi|  DEN|
| Silver| Hi-Hi|  DEN|
| Bronze|  Iran|  IRI|
| Bronze|  Iran|  IRI|
|   Gold|  Iran|  IRI|
| Silver|  Iran|  IRI|
| Bronze|  Iran|  IRI|
|   Gold|  Iran|  IRI|
| Bronze|  Iran|  IRI|
| Bronze|  Iran|  IRI|
| Silver|  Iran|  IRI|
| Silver|  Iran|  IRI|
| Bronze|  Iran|  IRI|
| Bronze|  Iran|  IRI|
| Bronze|  Iran|  IRI|
|   Gold|  Iran|  IRI|
| Bronze|  Iran|  IRI|
| Bronze|  Iran|  IRI|
| Silver|  Iran|  IRI|
| Bronze|  Iran|  IRI|
+-------+------+-----+
only showing top 20 rows



In [82]:
# Clase 17 Solucion reto joins
resultadosDF.filter(resultadosDF.medalla != "NA") \
.join(
    deportistaOlimpicoDF,
     resultadosDF.deportista_id == deportistaOlimpicoDF.deportista_id,
     "left"     
) \
.join(
    paisesDF,
     deportistaOlimpicoDF.equipo_id == paisesDF.pais_id,
     "left"
) \
.select("medalla", paisesDF.equipo, paisesDF.sigla) \
.sort(col("sigla").desc()) \
.show()

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



In [ ]:
Clase 18 Funciones de agrupación
En esta clase revisamos cuales son las funciones de agrupación con que cuenta Spark.

In [91]:
evento_schema = StructType([
    StructField('evento_id', IntegerType(),False),
    StructField('nombre', StringType(),False),
    StructField('deporte_id', IntegerType(),False)
]    
)

deportesOlimpicosDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(evento_schema).\
        load(path+"evento.csv")

deportesOlimpicosDF.show(4)

+---------+--------------------+----------+
|evento_id|              nombre|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
+---------+--------------------+----------+
only showing top 4 rows



In [96]:
#Clase 18 Funciones de agrupación
medallistaXanio = deportistaOlimpicoDF \
   .join (
       resultadosDF,
       deportistaOlimpicoDF.deportista_id == resultadosDF.deportista_id,
       "left"
         )\
   .join(
       juegoDF,
       juegoDF.juego_id == resultadosDF.juego_id,
       "left"
        )\
   .join(
     paisesDF,
     deportistaOlimpicoDF.equipo_id == paisesDF.pais_id,
     "left"
        )\
     .join(
         deportesOlimpicosDF,
         deportesOlimpicosDF.evento_id == resultadosDF.evento_id,
         "left"
        )\
      .join(
          deporteDF,
          deportesOlimpicosDF.deporte_id == deporteDF.deporte_id,
          "left"
          )\
       .select(
           "sigla",
           "anio",
           "medalla",
           deportesOlimpicosDF.nombre.alias("Nombre subdisciplina"),
           deporteDF.deporte.alias("Nombre disciplina"),
           deportistaOlimpicoDF.nombre
       )            


In [97]:
medallistaXanio.show()

+-----+------+-------+--------------------+-----------------+--------------------+
|sigla|  anio|medalla|Nombre subdisciplina|Nombre disciplina|              nombre|
+-----+------+-------+--------------------+-----------------+--------------------+
|  BRU|Bronze|     NA|Athletics Women's...|        Athletics|Maizurah Abdul Rahim|
|  BRU|  Gold|     NA|Shooting Men's Skeet|         Shooting|Jefri Kiko Bolkia...|
|  BRU|Bronze|     NA|Shooting Men's Skeet|         Shooting|Jefri Kiko Bolkia...|
|  BRU|  Gold|     NA|Athletics Men's 1...|        Athletics|     Jimmy Anak Ahar|
|  BRU|  Gold|     NA|Athletics Men's 1...|        Athletics|         Haseri Asli|
|  BRU|Bronze|     NA|Athletics Men's 1...|        Athletics|Mohamed Fakhri Is...|
|  BRU|  Gold|     NA|Swimming Men's 20...|         Swimming|Anderson Chee Wei...|
|  BRU|  Gold|     NA|Athletics Women's...|        Athletics|      Maziah Mahusin|
|  BRU|  Gold|     NA|Athletics Men's 4...|        Athletics|Ak Hafiy Tajuddin...|
|  B

In [102]:
#Descartamos deportistas sin medalla
medallistaXAnio2 = medallistaXanio.filter(resultadosDF.medalla != "NA")\
                                  .sort("anio")\
                                  .groupBy("sigla","anio","nombre subdisciplina")\
                                  .count() 
medallistaXAnio2.show()                                  

+-----+------+--------------------+-----+
|sigla|  anio|nombre subdisciplina|count|
+-----+------+--------------------+-----+
|  EUN|Bronze|Gymnastics Men's ...|    3|
|  EUN|  Gold|Athletics Women's...|    1|
|  UKR|  Gold|Canoeing Women's ...|    1|
|  FRA|Bronze|Fencing Women's e...|    3|
|  FRA|Bronze|Swimming Men's 4 ...|    6|
|  FRA|Bronze|Cycling Men's Mou...|    1|
|  FRA|  Gold|Tennis Women's Si...|    1|
|  RUS|  Gold|Cycling Women's P...|    2|
|  RUS|  Gold|Athletics Women's...|    1|
|  SUI|    NA|Fencing Men's epe...|   16|
|  FIN|    NA|Athletics Men's 1...|    4|
|  CRO|  Gold|Handball Men's Ha...|   30|
|  ROU|  Gold|Swimming Men's 20...|    1|
|  ROU|    NA|Wrestling Men's F...|    3|
|  ROU|    NA|Canoeing Men's Ca...|    1|
|  SLO|    NA|Snowboarding Men'...|    1|
|  ITA|    NA|Cross Country Ski...|    8|
|  ITA|Bronze|Shooting Men's Skeet|    3|
|  ITA|Bronze|Wrestling Men's W...|    1|
|  BUL|Bronze|Athletics Women's...|    2|
+-----+------+--------------------

In [103]:
medallistaXAnio2.printSchema()

root
 |-- sigla: string (nullable = true)
 |-- anio: string (nullable = true)
 |-- nombre subdisciplina: string (nullable = true)
 |-- count: long (nullable = false)



In [104]:
# .agg o agragate es la forma recomendada pos la documentación
medallistaXAnio2.groupBy("Sigla","anio")\
    .agg(sum("count").alias("Total de medallas"), \
     avg("count").alias("Medallas promedio")).show()

+-----+------+-----------------+------------------+
|Sigla|  anio|Total de medallas| Medallas promedio|
+-----+------+-----------------+------------------+
|  DJI|    NA|                1|               1.0|
|  ROU|  Gold|              102|              2.55|
|  ITA|Bronze|              223| 2.753086419753086|
|  CAN|    NA|              971| 5.678362573099415|
|  AZE|    NA|                7|               1.0|
|  ECU|Bronze|                1|               1.0|
|  BLR|Bronze|               29|1.6111111111111112|
|  JAM|Bronze|               51| 3.923076923076923|
|  TGA|Bronze|                1|               1.0|
|  EGY|  Gold|                7|               1.0|
|  THA|    NA|                7|1.1666666666666667|
|  MAR|Bronze|                6|               1.2|
|  BUL|    NA|              264| 3.142857142857143|
|  SCG|    NA|                9|              2.25|
|  TJK|    NA|                2|               1.0|
|  KEN|  Gold|               25|2.7777777777777777|
|  SWE|    N

Clase 19 SQL

En esta clase registramos DataFrames como si fueran SQL y veremos las diferencias entre ejecutar SQL como si fuera Spark y la forma nativa.

In [106]:
# registramos DataAframes como tables SQL
resultadosDF.registerTempTable("resultado")
deportistaOlimpicoDF.registerTempTable("deportista")
paisesDF.registerTempTable("paises")

In [107]:
sqlContext.sql("SELECT * FROM deportista").show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|            1|           A Dijiang|         24|      199|
|            2|            A Lamusi|         23|      199|
|            3| Gunnar Nielsen Aaby|         24|      273|
|            4|Edgar Lindenau Aabye|         34|      278|
|            5|Christine Jacoba ...|         21|      705|
+-------------+--------------------+-----------+---------+
only showing top 5 rows

